In [1]:
import os
import pandas as pd
import requests
import json
import requests
import multiprocessing as mp
from random import randint
from time import sleep

from rdkit import Chem

import sys
sys.path.insert(0,'/global/homes/b/bpb/repos/metatlas')
# from metatlas import metatlas_objects as metob

In [2]:
# df = pd.read_pickle('/project/projectdirs/openmsi/projects/ben_run_pactolus/unique_compounds.pkl')
df = pd.read_csv('/project/projectdirs/metatlas/projects/unique_compounds.csv.gz')
df.rename(columns = {'monoisotopoic_mw':'monoisotopic_mw'},inplace=True)

/global/common/software/m2650/python-cori/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1,2,5,6,7,10,11,12,13,14,15,25,26,28) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# all_compounds = metob.retrieve('Compounds',username = '*')

# all_compounds_df = metob.to_dataframe(all_compounds)


# all_compounds_df.keys()

In [4]:
# df = pd.read_pickle('/project/projectdirs/openmsi/projects/ben_run_pactolus/unique_compounds.pkl')
# # df = pd.read_csv('/project/projectdirs/openmsi/projects/compound_data/jgi_molecules/new_jgi_compounds.csv')
# df.rename(columns = {'monoisotopoic_mw':'monoisotopic_mw'},inplace=True)

In [5]:
df = df.convert_objects(convert_numeric=True)
df.keys()

/global/common/software/m2650/python-cori/lib/python2.7/site-packages/IPython/kernel/__main__.py:1: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  if __name__ == '__main__':


Index([u'Unnamed: 0', u'chebi_id', u'chebi_url', u'description', u'formula',
       u'hmdb_id', u'hmdb_url', u'img_abc_id', u'inchi', u'inchi_key',
       u'iupac_name', u'kegg_id', u'kegg_url', u'lipidmaps_id',
       u'lipidmaps_url', u'metacyc_id', u'mono_isotopic_molecular_weight',
       u'name', u'neutralized_2d_inchi', u'neutralized_2d_inchi_key',
       u'neutralized_inchi', u'neutralized_inchi_key', u'num_free_radicals',
       u'number_components', u'permanent_charge', u'pubchem_compound_id',
       u'pubchem_url', u'source', u'synonyms', u'wikipedia_url'],
      dtype='object')

In [6]:
inchi_keys = df[(~ pd.isnull(df.inchi_key))].inchi_key

In [7]:
out_dir = '/project/projectdirs/metatlas/projects/pubchem_info/'
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

# remove those that have already been done

In [8]:
import glob
files = glob.glob(os.path.join(out_dir,'*.json'))
done_inchi_key = []
for f in files:
    done_inchi_key.append(os.path.basename(f).split('.')[0])
inchi_keys = list(set(inchi_keys) - set(done_inchi_key))

In [9]:
len(inchi_keys)

196656

In [10]:
# chunks = [inchi_keys[x:x+1000] for x in xrange(0, len(inchi_keys), 1000)]


In [11]:
def write_pubchem_info_to_file(ik):
    suffix = '.json'
    fname = os.path.join(out_dir, ik + suffix)
    # this query will return 
    # {"Fault": {"Message": "No records found for the given CID(s)", "Code": "PUGREST.NotFound"}}
    # if 3d data is not available from pubchem.
    # leaving it out does not get you 3d. It defaults to 2d if left out.  Have to run it twice.
    #
    # missing inchikeys will return
    # {"Fault": {"Message": "No CID found", "Code": "PUGREST.NotFound", "Details": ["No CID found that matches the given InChI key"]}}
    #
    url = 'https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/inchikey/%s/json?record_type=3d'%ik
    response = requests.get(url)
    d = response.json()
    if 'Fault' in d.keys():
        url = 'https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/inchikey/%s/json'%ik
        response = requests.get(url)
        d = response.json()
    if "Fault" in d.keys():
        with open(fname, 'w') as fid:
            fid.write('ERROR')
    else:
        with open(fname, 'w') as fid:
            json.dump(d, fid)

            

#     url = 'https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/inchikey/%s/json?record_type=3d'%inchi_key
#     try: 
#         response = requests.get(url)
#         with open(fname, 'w') as fid:
#             json.dump(response.json(), fid)
#     except:
#         print "could not query", inchi_key

In [12]:
for ik in inchi_keys:
    write_pubchem_info_to_file(ik)

KeyboardInterrupt: 

In [ ]:
# pool = mp.Pool(processes=20)
# pool.map(write_pubchem_info_to_file, inchi_keys)

In [13]:
# ik = 'JAISCAWSAHCRJE-GJKQCQSZSA-N' #this one is only 2d in pubchem
# ik = 'PARFVHOUOBIHJG-GSHUGGBRSA-N' #this one has 2d and 3d in pubchem
# ik = "NZPACTGCRWDXCJ-UHFFFAOYSA-N" #this one is not in pubchem
ik = 'AMVODTGMYSRMNP-GNIMZFFESA-N' #this one is messed up for some reason
url = 'https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/inchikey/%s/json?record_type=3d'%ik
print url
response = requests.get(url)
d = response.json()
print d.keys()
if 'Fault' in d.keys():
    url = 'https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/inchikey/%s/json'%ik
    print url
    response = requests.get(url)
    d = response.json()
    print d.keys()

https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/inchikey/AMVODTGMYSRMNP-GNIMZFFESA-N/json?record_type=3d
[u'PC_Compounds']


In [ ]:
# response.json()

In [ ]:
# url = 'https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/%s/json'%'Baumin'
# response = requests.get(url)
# response.json()